<a href="https://colab.research.google.com/github/BharathkumarK19/Resume_projects/blob/main/Image_Classification_Project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/

cp: cannot stat 'kaggle.json': No such file or directory


In [ ]:
!pip install keras-tuner -q #Installs Keras Tuner
import keras_tuner as kt #Imports it to our notebook

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 129.1/129.1 kB 10.3 MB/s eta 0:00:00


In [ ]:
!pip install opendatasets

In [ ]:
import opendatasets as od
import  pandas as pd

In [ ]:
od.download('https://www.kaggle.com/datasets/pavansanagapati/images-dataset')

Please provide your Kaggle credentials to download this dataset. Learn more: http://bit.ly/kaggle-creds
Your Kaggle username: "bharathkaranam45"
Your Kaggle Key: ··········
Dataset URL: https://www.kaggle.com/datasets/pavansanagapati/images-dataset


100%|██████████| 1.01G/1.01G [00:51<00:00, 21.1MB/s]


In [ ]:
import tensorflow as tf
from tensorflow import keras
from keras.layers import Dense,Conv2D,MaxPooling2D,Flatten,BatchNormalization,Dropout
from keras.models import Sequential
from keras.applications.vgg16 import VGG16

In [ ]:
con_base=VGG16(
    weights='imagenet',
    include_top=False,
    input_shape=(150,150,3)
    )

58889256/58889256 ━━━━━━━━━━━━━━━━━━━━ 4s 0us/step


In [ ]:
data=keras.utils.image_dataset_from_directory(
    directory='/content/images-dataset/data',
    labels='inferred',
    label_mode='int',
    batch_size=32,
    image_size=(150,150)
)

Found 3606 files belonging to 8 classes.


In [ ]:
con_base.trainable = True  # First, make sure the entire model is trainable

for layer in con_base.layers:
    if layer.name.startswith('block5') or layer.name.startswith('block4'):
        layer.trainable = True  # Unfreeze layers in block4 and block5
    else:
        layer.trainable = False  # Freeze all other layers

In [ ]:
for layer1 in con_base.layers:
  print(layer1.name,layer1.trainable)

input_layer False
block1_conv1 False
block1_conv2 False
block1_pool False
block2_conv1 False
block2_conv2 False
block2_pool False
block3_conv1 False
block3_conv2 False
block3_conv3 False
block3_pool False
block4_conv1 True
block4_conv2 True
block4_conv3 True
block4_pool True
block5_conv1 True
block5_conv2 True
block5_conv3 True
block5_pool True


In [ ]:
import tensorflow as tf
dataset_size = tf.data.experimental.cardinality(data).numpy()
train_split = 0.8
train_size = int(train_split * dataset_size)
train_ds = data.take(train_size)
test_ds = data.skip(train_size)

In [ ]:
import tensorflow as tf

def process(image, label):
  image = tf.cast(image, tf.float32) / 255.0
  # One-hot encode the label
  label = tf.keras.utils.to_categorical(label, num_classes=8)  # Assuming you have 8 classes
  return image, label

train_ds = train_ds.map(process)
test_ds = test_ds.map(process)

In [ ]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

train_datagen = ImageDataGenerator(
    rescale=1./255,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    width_shift_range=0.1,
    height_shift_range=0.1,
    rotation_range=40,
)

train_generator = train_datagen.flow_from_directory(
    '/content/images-dataset/data',
    target_size=(150, 150),
    batch_size=32,
    class_mode='categorical'
)

In [ ]:
def build_model(hp):
    model = Sequential()
    model.add(con_base)
    model.add(Flatten())
    model.add(BatchNormalization())
    model.add(Dense(units=hp.Int('units', min_value=32, max_value=512, step=32),
                    activation='relu', kernel_regularizer=keras.regularizers.l2(hp.Choice('reg_rate', values=[1e-2, 1e-3, 1e-4]))))
    model.add(Dropout(hp.Float('dropout_rate', min_value=0.0, max_value=0.5, step=0.1)))
    model.add(Dense(256,activation='relu',kernel_regularizer=keras.regularizers.l2(0.01)))
    model.add(Dropout(0.2))
    model.add(BatchNormalization())
    model.add(Dense(85,activation='relu',kernel_regularizer=keras.regularizers.l2(0.01)))
    model.add(Dropout(0.2))
    model.add(BatchNormalization())
    model.add(Dense(30,activation='relu',kernel_regularizer=keras.regularizers.l2(0.01)))
    model.add(Dense(8,activation='softmax'))
    # ... (rest of your model architecture)

    model.compile(optimizer=keras.optimizers.Adam(hp.Choice('learning_rate', values=[1e-2, 1e-3, 1e-4])),
                  loss='categorical_crossentropy',
                  metrics=['accuracy'])
    return model

tuner = kt.Hyperband(build_model,
                     objective='val_accuracy',
                     max_epochs=10,
                     factor=3,
                     directory='my_dir',
                     project_name='intro_to_kt')

tuner.search(train_generator, epochs=10, validation_data=test_ds)

best_hps=tuner.get_best_hyperparameters(num_trials=1)[0]

model = tuner.hypermodel.build(best_hps)

Trial 30 Complete [00h 06m 55s]
val_accuracy: 0.5110192894935608

Best val_accuracy So Far: 0.5192837715148926
Total elapsed time: 01h 29m 53s
